In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [18]:
%pip uninstall torch

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3.10/shutil.py", line 816, in move
    os.rename(src, real_dst)
PermissionError: [Errno 13] Permission denied: '/usr/local/bin/convert-caffe2-to-onnx' -> '/tmp/pip-uninstall-0g6d1y2g/convert-caffe2-to-onnx'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/uninstall.py", line 105, in run
    uninstall_pathset = req.uninstall(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_install.py", line 687, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_uninstall.py", 

In [73]:
config = PeftConfig.from_pretrained('./algorithms-genius/')

In [93]:
import torch
model = AutoModelForCausalLM.from_pretrained(
    "sharpbai/Llama-2-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    './algorithms-genius/',
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/34 [00:00<?, ?it/s]

In [94]:
model.eval()
generation_config = GenerationConfig.from_pretrained("sharpbai/Llama-2-7b-hf")
print(generation_config)

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.9,
  "top_p": 0.6,
  "transformers_version": "4.30.0"
}



In [95]:
tokenizer = AutoTokenizer.from_pretrained("sharpbai/Llama-2-7b-hf")

In [96]:
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Imagine that you are a robot assistant with one mechanical arm. You can only execute 3 commands that take arguments: 'where' is the place where the action needs to be performed, 'what' is the object on which the action needs to be performed. List of available commands:\nmove to command - moves to 'where' or 'what'.\nThe 'pick up' command is to take the object 'what' with a mechanical hand from the place 'where'.\nThe put command is to place the 'what' object from the hand to the 'where' location.\nif the user’s request does not specify the place 'where', replace it with the word 'unspecified'."
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_token_type_ids=False, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [112]:
!ln -s libnvrtc-builtins.so  libnvrtc-builtins.so.12.1

ln: failed to create symbolic link 'libnvrtc-builtins.so.12.1': File exists


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:297 in <lambda>                         │
│                                                                                                  │
│   294 │   │   self.shell.register_magics(livyMagics)                                             │
│   295 │   │                                                                                      │
│   296 │   │   self._script_executor = ScriptExecutor(get_output_error_code)                      │
│ ❱ 297 │   │   self.shell.system = lambda code: self._script_executor.execute("bash", code)       │
│   298 │                                                                                          │
│   299 │   def _init_message_handlers(self):                                                      │
│   300 │   │   self.log.debug('MLKernel._init_message_handlers()')                                │
│                                                                                                  │
│ /kernel/lib/python3.10/site-packages/ml_kernel/script_executor.py:43 in execute                  │
│                                                                                                  │
│   40 │   │                                                                                       │
│   41 │   │   return_code = self.do_execute(code)                                                 │
│   42 │   │   if return_code != 0:                                                                │
│ ❱ 43 │   │   │   raise Exception('Process exited with code %d' % return_code)                    │
│   44                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: Process exited with code 1

/bin/sh: 10: ll: not found


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:297 in <lambda>                         │
│                                                                                                  │
│   294 │   │   self.shell.register_magics(livyMagics)                                             │
│   295 │   │                                                                                      │
│   296 │   │   self._script_executor = ScriptExecutor(get_output_error_code)                      │
│ ❱ 297 │   │   self.shell.system = lambda code: self._script_executor.execute("bash", code)       │
│   298 │                                                                                          │
│   299 │   def _init_message_handlers(self):                                                      │
│   300 │   │   self.log.debug('MLKernel._init_message_handlers()')                                │
│                                                                                                  │
│ /kernel/lib/python3.10/site-packages/ml_kernel/script_executor.py:43 in execute                  │
│                                                                                                  │
│   40 │   │                                                                                       │
│   41 │   │   return_code = self.do_execute(code)                                                 │
│   42 │   │   if return_code != 0:                                                                │
│ ❱ 43 │   │   │   raise Exception('Process exited with code %d' % return_code)                    │
│   44                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: Process exited with code 127

In [123]:
!ls -l /usr/local/cuda/lib64/libnvrtc*

lrwxrwxrwx 1 root root       25 Sep 21  2022 /usr/local/cuda/lib64/libnvrtc-builtins.so -> libnvrtc-builtins.so.11.8
lrwxrwxrwx 1 root root       28 Sep 21  2022 /usr/local/cuda/lib64/libnvrtc-builtins.so.11.8 -> libnvrtc-builtins.so.11.8.89
-rw-r--r-- 1 root root  7718792 Sep 21  2022 /usr/local/cuda/lib64/libnvrtc-builtins.so.11.8.89
-rw-r--r-- 1 root root  7743142 Sep 21  2022 /usr/local/cuda/lib64/libnvrtc-builtins_static.a
lrwxrwxrwx 1 root root       16 Sep 21  2022 /usr/local/cuda/lib64/libnvrtc.so -> libnvrtc.so.11.2
lrwxrwxrwx 1 root root       19 Sep 21  2022 /usr/local/cuda/lib64/libnvrtc.so.11.2 -> libnvrtc.so.11.8.89
-rw-r--r-- 1 root root 54409816 Sep 21  2022 /usr/local/cuda/lib64/libnvrtc.so.11.8.89
-rw-r--r-- 1 root root 71382614 Sep 21  2022 /usr/local/cuda/lib64/libnvrtc_static.a


In [113]:
inputs = ["Take the white toy from the table and take it to the orange box."]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│ in generate:49                                                                                   │
│                                                                                                  │
│ /home/jupyter/.local/lib/python3.10/site-packages/peft/peft_model.py:711 in generate             │
│                                                                                                  │
│    708 │   │   self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generati  │
│    709 │   │   try:                                                                              │
│    710 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  711 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    712 │   │   │   else:                                                                         │
│    713 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    714 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ in decorate_context:115                                                                          │
│                                                                                                  │
│ /home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1522 in       │
│ generate                                                                                         │
│                                                                                                  │
│   1519 │   │   │   │   )                                                                         │
│   1520 │   │   │                                                                                 │
│   1521 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1522 │   │   │   return self.greedy_search(                                                    │
│   1523 │   │   │   │   input_ids,                                                                │
│   1524 │   │   │   │   logits_processor=logits_processor,                                        │
│   1525 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:2392 in       │
│ greedy_search                                                                                    │
│                                                                                                  │
│   2389 │   │   │   # if eos_token was found in one sentence, set sentence to finished            │
│   2390 │   │   │   if eos_token_id_tensor is not None:                                           │
│   2391 │   │   │   │   unfinished_sequences = unfinished_sequences.mul(                          │
│ ❱ 2392 │   │   │   │   │   next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_te  │
│   2393 │   │   │   │   )                                                                         │
│   2394 │   │   │   │                                                                             │
│   2395 │   │   │   │   # stop when each sentence is finished                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: 
  #ifdef __HIPCC__
  #define ERROR_UNSUPPORTED_CAST ;
  // corresponds to aten/src/ATen/native/cuda/thread_constants.h
  #define CUDA_OR_ROCM_NUM_T